In [ ]:
!pip install xlrd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os, fnmatch
from tqdm import tqdm

In [ ]:
# Constant
ESP_final_path_folder = "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ESP"
ITA_final_path_folder = "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ITA"
GRC_final_path_folder = "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/GRC"

climate_path_folder = "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned"

## Inflow dataset

In [ ]:
ESP_list_inflow = [
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ESP/FINAL Q4 2017 Arrivals_to_Europe - ESP.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ESP/FINAL Q4 2018 Arrivals_to_Europe - ESP.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ESP/FINAL Q4 2019 Arrivals_to_Europe - ESP.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ESP/FINAL Q4 2020 Arrivals_to_Europe - ESP.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ESP/FINAL Q4 2021 Arrivals_to_Europe - ESP.xlsx"
]
ITA_list_inflow = [
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ITA/FINAL Q4 2017 Arrivals_to_Europe - ITA.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ITA/FINAL Q4 2018 Arrivals_to_Europe - ITA.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ITA/FINAL Q4 2019 Arrivals_to_Europe - ITA.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ITA/FINAL Q4 2020 Arrivals_to_Europe - ITA.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/ITA/FINAL Q4 2021 Arrivals_to_Europe - ITA.xlsx"
]
GRC_list_inflow = [
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/GRC/FINAL Q4 2017 Arrivals_to_Europe - GRC.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/GRC/FINAL Q4 2018 Arrivals_to_Europe - GRC.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/GRC/FINAL Q4 2019 Arrivals_to_Europe - GRC.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/GRC/FINAL Q4 2020 Arrivals_to_Europe - GRC.xlsx",
    "/content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Inflows dataset/GRC/FINAL Q4 2021 Arrivals_to_Europe - GRC.xlsx"
]

### Search on climate dataset

In [ ]:
# Find file by regex and path
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [ ]:
def put_climate_data(iso, type_of_climate, name_of_column, df):

  file = find(str(iso) + '*.csv', str(climate_path_folder) + "/" + str(type_of_climate))
  if len(file) > 0:
    file = file[0]
  else:
    print(f'File non esistente: {str(climate_path_folder) + "/" + str(type_of_climate) + "/" + str(iso) + "*.csv"}')
    return

  df_climate = pd.read_csv(file)

  df_climate = df_climate[df_climate.columns.drop(list(df_climate.filter(regex='Unnamed')))]
  df_climate['ds'] = pd.to_datetime(df_climate['ds'], format='%Y-%m-%d')
  df_climate['ds'] = df_climate['ds'].dt.strftime('%Y-%m')

  for index, row in df_climate.iterrows():
    date = row["ds"] 
    query = "`Reported_Date` == @date and `ISO3_of_Origin` == @iso"
    df.loc[df.eval(query), name_of_column] = row["y"]

### SPAGNA

In [ ]:
final_df = pd.DataFrame()

for esp_inflow in tqdm(ESP_list_inflow):
  df = pd.read_excel(esp_inflow)
  df.columns = df.columns.str.replace(' ', '_')
  df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
  # Set type of data
  df['Reported_Date'] = df['Reported_Date'].dt.strftime('%Y-%m')
  df = df.sort_values(by=['ISO3_of_Origin', 'Reported_Date'])

  # Add climate information
  for iso in df['ISO3_of_Origin'].unique():
    put_climate_data(iso, 'Min temperature', 'Min_T', df)
    put_climate_data(iso, 'Max temperature', 'Max_T', df)
    put_climate_data(iso, 'Mean temperature', 'Avg_T', df)
    put_climate_data(iso, 'Percipitation', 'mm_Percip', df)
  
  # Here other useful informations (i.e: welfare info, etc...)

  # Merge dfs into only one df
  final_df = pd.concat([final_df, df])
  print(f'dataset length: {len(final_df)}')

# Save file to drive
with open(ESP_final_path_folder + "/MERGED.csv", 'w', encoding='utf-8-sig') as f:
  final_df.to_csv(f)

  0%|          | 0/5 [00:00<?, ?it/s]

File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/OOO*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Max temperature/OOO*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Mean temperature/OOO*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Percipitation/OOO*.csv


 20%|██        | 1/5 [16:22<1:05:28, 982.12s/it]

dataset length: 54
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/OOS*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Max temperature/OOS*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Mean temperature/OOS*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Percipitation/OOS*.csv


 40%|████      | 2/5 [30:27<45:05, 901.80s/it]  

...

dataset length: 794
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/OOO*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Max temperature/OOO*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Mean temperature/OOO*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Percipitation/OOO*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/OOS*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Ap

 80%|████████  | 4/5 [54:18<12:52, 772.31s/it]

File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Max temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Mean temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Percipitation/nan*.csv
dataset length: 908
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/OOS*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Ap

100%|██████████| 5/5 [1:04:31<00:00, 774.27s/it]

dataset length: 1059


### ITA

In [ ]:
final_it_df = pd.DataFrame()

for ita_inflow in tqdm(ITA_list_inflow):
  df = pd.read_excel(ita_inflow)
  df.columns = df.columns.str.replace(' ', '_')
  df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
  # Set type of data
  df['Reported_Date'] = df['Reported_Date'].dt.strftime('%Y-%m')
  df = df.sort_values(by=['ISO3_of_Origin', 'Reported_Date'])

  # Add climate information
  for iso in df['ISO3_of_Origin'].unique():
    put_climate_data(iso, 'Min temperature', 'Min_T', df)
    put_climate_data(iso, 'Max temperature', 'Max_T', df)
    put_climate_data(iso, 'Mean temperature', 'Avg_T', df)
    put_climate_data(iso, 'Percipitation', 'mm_Percip', df)
  
  # Here other useful informations (i.e: welfare info, etc...)

  # Merge dfs into only one df
  final_it_df = pd.concat([final_it_df, df])
  print(f'dataset length: {len(final_it_df)}')
  
# Save file to drive
with open(ITA_final_path_folder + "/ITA_MERGED.csv", 'w', encoding='utf-8-sig') as f:
  final_it_df.to_csv(f)

 20%|██        | 1/5 [04:35<18:20, 275.17s/it]

File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Max temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Mean temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Percipitation/nan*.csv
dataset length: 134


 40%|████      | 2/5 [08:32<12:38, 252.92s/it]

File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Max temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Mean temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Percipitation/nan*.csv
dataset length: 278


/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


...

dataset length: 667


 80%|████████  | 4/5 [47:56<14:18, 858.21s/it]

File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Max temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Mean temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Percipitation/nan*.csv
dataset length: 1130


100%|██████████| 5/5 [1:05:45<00:00, 789.00s/it]

File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Max temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Mean temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Percipitation/nan*.csv
dataset length: 1623


### GRECIA

In [ ]:
final_grc_df = pd.DataFrame()

for grc_inflow in tqdm(GRC_list_inflow):
  df = pd.read_excel(grc_inflow)
  df.columns = df.columns.str.replace(' ', '_')
  df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
  # Set type of data
  df['Reported_Date'] = df['Reported_Date'].dt.strftime('%Y-%m')
  df = df.sort_values(by=['ISO3_of_Origin', 'Reported_Date'])

  # Add climate information
  for iso in df['ISO3_of_Origin'].unique():
    put_climate_data(iso, 'Min temperature', 'Min_T', df)
    put_climate_data(iso, 'Max temperature', 'Max_T', df)
    put_climate_data(iso, 'Mean temperature', 'Avg_T', df)
    put_climate_data(iso, 'Percipitation', 'mm_Percip', df)
  
  # Here other useful informations (i.e: welfare info, etc...)

  # Merge dfs into only one df
  final_grc_df = pd.concat([final_grc_df, df])
  print(f'dataset length: {len(final_grc_df)}')
  
# Save file to drive
with open(GRC_final_path_folder + "/GRC_MERGED.csv", 'w', encoding='utf-8-sig') as f:
  final_grc_df.to_csv(f)

 20%|██        | 1/5 [22:36<1:30:25, 1356.29s/it]

File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Max temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Mean temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Percipitation/nan*.csv
dataset length: 722
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/XKX*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Ap

 40%|████      | 2/5 [40:45<59:57, 1199.29s/it]  

File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Min temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Max temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Mean temperature/nan*.csv
File non esistente: /content/drive/MyDrive/University /Polito Master/Applied data science project/Project/Datasets/Climate datasets/Climate_datasets_Cleaned/Percipitation/nan*.csv
dataset length: 1258


 60%|██████    | 3/5 [1:00:37<39:51, 1195.73s/it]

dataset length: 1721
